In [ ]:
import numpy as np
import cv2
import os
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
model = nn.Sequential(
    nn.Linear(784, 16, bias=False),
    nn.BatchNorm1d(16),
    nn.Sigmoid(),
	nn.Linear(16, 1, bias=False),
	nn.Sigmoid()
).double()

In [ ]:
HW_dir = ".."
# input data
train_data = np.zeros((9,784)) 
for i in range(9):
    train_data[i,:] = cv2.imread(os.path.join(HW_dir, "mnist_subset", f"{i}.png"), cv2.IMREAD_GRAYSCALE).reshape(-1)/255.
gt_y = np.zeros((9,1)) 
gt_y[0] =1  
val_data = np.zeros((1,784)) 
val_data[0,:] = cv2.imread(os.path.join(HW_dir, "mnist_subset", "9.png"), cv2.IMREAD_GRAYSCALE).reshape(-1)/255.
val_gt = np.zeros((1,1)) 
np.random.seed(14)

# Intialize MLP  (784 -> 16 -> 1)
MLP_layer_1 = np.random.randn(784,16)
MLP_layer_2 = np.random.randn(16,1)
model[0].weight.data = torch.tensor(MLP_layer_1.T, dtype=torch.double)
model[3].weight.data = torch.tensor(MLP_layer_2.T, dtype=torch.double)

# Initialize gamma and beta
gamma = np.random.randn(16)
beta = np.random.randn(16)
model[1].weight.data = torch.tensor(gamma, dtype=torch.double)
model[1].bias.data = torch.tensor(beta, dtype=torch.double)
#model[1].running_var.data = torch.tensor(np.zeros(16), dtype=torch.double)

lr=1e-1
loss_list=[]
# ---------------- TODO -------------------
# compute mean and var for testing
# add codes anywhere as you need
# ---------------- TODO -------------------
mean = np.zeros(16, dtype=np.float64)
var = np.zeros(16, dtype=np.float64)
momentum = 0.1

optim = torch.optim.SGD(model.parameters(), lr=lr)
activation_list = []

# training 
gt_y = torch.tensor(gt_y, dtype=torch.double)
for i in range(50):
	# Forward
	print("here 0: ", model[1].running_var.data.detach().numpy())

	pred_y = model(torch.tensor(train_data, dtype=torch.double))

	print("here 1: ", model[1].running_var.data.detach().numpy())
	with torch.no_grad():
		activation = model[0](torch.tensor(train_data, dtype=torch.double))
		activation_list.append(activation.detach().numpy())
		print("here 2: ", activation.var(0))

	# compute loss
	loss = -( gt_y * torch.log(pred_y) + (1-gt_y) * torch.log(1-pred_y)).sum()
	print("iteration: %d, loss: %f" % (i+1 ,loss.data.detach().numpy()))
	loss_list.append(loss.data.detach().numpy())
     
    # Backward : compute the gradient of paratmerters of layer1 (grad_layer_1) and layer2 (grad_layer_2)
	optim.zero_grad()
	loss.backward()
	optim.step()

# validate
model.eval()
val_gt = torch.tensor(val_gt, dtype=torch.double)
pred_y = model(torch.tensor(val_data, dtype=torch.double))
loss = -( val_gt * torch.log(pred_y) + (1-val_gt) * torch.log(1-pred_y)).sum()
print("validation loss: %f" % (loss))
loss_list.append(loss)

In [ ]:
import numpy as np
import cv2
import os


# eps may help you to deal with numerical problem
eps = 1e-5
def bn_forward_test(x, gamma, beta, mean, var):

    #----------------TODO------------------
    # Implement forward 
    #----------------TODO------------------
    x_hat = (x - mean) / np.sqrt(var + eps)
    out = gamma * x_hat + beta    

    return out

def bn_forward_train(x, gamma, beta):

    #----------------TODO------------------
    # Implement forward 
    #----------------TODO------------------

    # save intermidiate variables for computing the gradient when backward
    sample_mean = np.mean(x, axis=0)
    sample_var = np.var(x, axis=0)
    x_hat = (x - sample_mean) / np.sqrt(sample_var + eps)
    out = gamma * x_hat + beta

    cache = (gamma, x, x_hat, sample_mean, sample_var)
    return out, cache
    
def bn_backward(dout, cache):

    #----------------TODO------------------
    # Implement backward 
    #----------------TODO------------------
    gamma, x, x_hat, sample_mean, sample_var = cache
    dgamma = np.sum(dout * x_hat, axis=0)
    dbeta = np.sum(dout, axis=0)
    dx_hat = dout * gamma

    N, D = x.shape
    dx = (1.0 / (N * np.sqrt(sample_var + eps))) * (N * dx_hat - np.sum(dx_hat, axis=0) - x_hat*np.sum(dx_hat*x_hat, axis=0))

    return dx, dgamma, dbeta

# This function may help you to check your code
def print_info(x):
    print('mean:', np.mean(x,axis=0))
    print('var:',np.var(x,axis=0))
    print('------------------')
    return 

def train():
	HW_dir = ".."

	# input data
	train_data = np.zeros((9,784)) 
	for i in range(9):
		train_data[i,:] = cv2.imread(os.path.join(HW_dir, "mnist_subset", f"{i}.png"), cv2.IMREAD_GRAYSCALE).reshape(-1)/255.
	gt_y = np.zeros((9,1)) 
	gt_y[0] =1  

	val_data = np.zeros((1,784)) 
	val_data[0,:] = cv2.imread(os.path.join(HW_dir, "mnist_subset", "9.png"), cv2.IMREAD_GRAYSCALE).reshape(-1)/255.
	val_gt = np.zeros((1,1)) 

	np.random.seed(14)

	# Intialize MLP  (784 -> 16 -> 1)
	MLP_layer_1 = np.random.randn(784,16)
	MLP_layer_2 = np.random.randn(16,1)

	# Initialize gamma and beta
	gamma = np.random.randn(16)
	beta = np.random.randn(16)

	lr=1e-1
	loss_list=[]

	# ---------------- TODO -------------------
	# compute mean and var for testing
	# add codes anywhere as you need
	# ---------------- TODO -------------------
	mean = np.zeros(16, dtype=np.float64)
	var = np.ones(16, dtype=np.float64)
	momentum = 0.1
    
	sample_mean_list = []
	mean_list = []
	var_list = []
	activation_list = []

	# training 
	for i in range(50):
		# Forward
		output_layer_1 = train_data.dot(MLP_layer_1)
		activation_list.append(output_layer_1)
		output_layer_1_bn, cache = bn_forward_train(output_layer_1, gamma, beta)
		output_layer_1_act = 1 / (1+np.exp(-output_layer_1_bn))  #sigmoid activation function
		output_layer_2 = output_layer_1_act.dot(MLP_layer_2)
		pred_y = 1 / (1+np.exp(-output_layer_2))  #sigmoid activation function

		# update running mean and var
		B = train_data.shape[0]
		mean = (1-momentum) * mean + momentum * (cache[-2])
		#print("here 0: ", var)
		var = (1-momentum) * var + momentum * (cache[-1]*B/(B-1))
		#print("here 1: ", cache[-1])
		#print("here 2: ", var)

		mean_list.append(mean)
		var_list.append(var)
		sample_mean_list.append(cache[-2])

		# compute loss 
		loss = -( gt_y * np.log(pred_y) + (1-gt_y) * np.log(1-pred_y)).sum()
		print("iteration: %d, loss: %f" % (i+1 ,loss))
		loss_list.append(loss)

		# Backward : compute the gradient of paratmerters of layer1 (grad_layer_1) and layer2 (grad_layer_2)
		grad_pred_y = -(gt_y/pred_y) + (1-gt_y)/(1-pred_y)
		grad_activation_func = grad_pred_y * pred_y * (1-pred_y) 
		grad_layer_2 = output_layer_1_act.T.dot(grad_activation_func)
		grad_output_layer_1_act = grad_activation_func.dot(MLP_layer_2.T)
		grad_output_layer_1_bn  = grad_output_layer_1_act * (1-output_layer_1_act) * output_layer_1_act
		grad_output_layer_1, grad_gamma, grad_beta = bn_backward(grad_output_layer_1_bn, cache)
		grad_layer_1 = train_data.T.dot(grad_output_layer_1)

		# update parameters
		gamma -= lr * grad_gamma
		beta -= lr * grad_beta
		MLP_layer_1 -= lr * grad_layer_1
		MLP_layer_2 -= lr * grad_layer_2

	# validate
	output_layer_1 = val_data.dot(MLP_layer_1)
	output_layer_1_bn = bn_forward_test(output_layer_1, gamma, beta, mean, var)
	output_layer_1_act = 1 / (1+np.exp(-output_layer_1_bn))  #sigmoid activation function
	output_layer_2 = output_layer_1_act.dot(MLP_layer_2)
	pred_y = 1 / (1+np.exp(-output_layer_2))  #sigmoid activation function
	loss = -( val_gt * np.log(pred_y) + (1-val_gt) * np.log(1-pred_y)).sum()
	print("validation loss: %f" % (loss))
	loss_list.append(loss)
      
	return [output_layer_1, output_layer_1_bn, output_layer_1_act, output_layer_2, pred_y], mean_list, var_list, activation_list, gamma, beta

In [ ]:
output, mean_list, var_list, activation_list, gamma, beta =train()

In [ ]:
output_2 = []
with torch.no_grad():
    for i in range(5):
        m = model[:i+1]
        output_2.append(m(torch.tensor(val_data, dtype=torch.double)))

In [ ]:
print(torch.var(torch.tensor(activation_list[0]), dim=0)*8/9)
"""
[ 6.84644861  4.75433697 10.10462936  3.77963637  7.4597243   2.30848636
  5.0162455   6.82881662 10.99939121  4.89221989  4.70891663  9.18769918
  5.05513413  4.05783713  2.68460886  9.2198289 ]
"""

In [ ]:
np.allclose(model[1].running_var.data.detach().numpy(), var_list[-1])

In [ ]:
np.random.seed(14)
# Initialize gamma and beta
gamma = np.random.randn(16, )
beta = np.random.randn(16)

In [ ]:
x = np.random.rand(3, 16)
out, cache = bn_forward_train(x, gamma, beta)
dout = np.random.rand(3, 16)
dx, dgamma, dbeta = bn_backward(dout, cache)

In [ ]:
x_tensor = torch.tensor(x, requires_grad=True, dtype=torch.double)
m = nn.BatchNorm1d(num_features=x_tensor.shape[1]).double()
m.weight.data = torch.tensor(gamma, dtype=torch.double)
m.bias.data = torch.tensor(beta, dtype=torch.double)
y = m(x_tensor)
y.retain_grad()
loss = torch.sum(y * torch.tensor(dout))
loss.backward()

In [ ]:
## compare parameters
print("gamma:")
print(gamma)
print("beta:")
print(beta)

for p in m.parameters():
	print(p.data.detach().numpy())

In [ ]:
## compare outputs
print(out)
print(y.data.detach().numpy())
print(np.allclose(out, y.data.detach().numpy()))

In [ ]:
## compare grads
print("dout:")
print(dout)
print("y.grad:")
print(y.grad)
print(np.allclose(dout, y.grad.data.detach().numpy()))

In [ ]:
## compare parameters grad
print("dgamma:")
print(dgamma)
print("dbeta:")
print(dbeta)

dparams = []
for p in m.parameters():
	dparams.append(p.grad.data.detach().numpy())
	print(p.grad.data.detach().numpy())

print(np.allclose(dgamma, dparams[0]))
print(np.allclose(dbeta, dparams[1]))

In [ ]:
## compare input grad
print("dx:")
print(dx)
print("x_tensor.grad:")
print(x_tensor.grad.data.detach().numpy())
print(np.allclose(dx, x_tensor.grad.data.detach().numpy()))